In [1]:
import random
from typing import Optional

import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import models, datasets, transforms

from aimet_torch.adaround.adaround_weight import AdaroundParameters
from aimet_torch.auto_quant import AutoQuant
# from aimet_torch.auto_quant_v2 import AutoQuant

import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

2025-02-26 17:44:56,975 - root - INFO - AIMET


/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/usr/lib/python3.10/abc.py:106: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.
  cls = super().__new__(mcls, name, bases, namespace, **kwargs)
/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LightningModel(pl.LightningModule):
    def __init__(self, num_classes=64, learning_rate=2e-5):
        super(LightningModel, self).__init__()
        self.model = models.mobilenet_v3_small(pretrained=True)
        self.model.classifier[3] = nn.Linear(self.model.classifier[3].in_features, num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        images, labels = images.to(self.device), labels.to(self.device)
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)


In [3]:


# Step 1. Define constants and helper functions
EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 2000
BATCH_SIZE = 32

_subset_samplers = {}

def _create_sampled_data_loader(dataset, num_samples):
    if num_samples not in _subset_samplers:
        indices = random.sample(range(len(dataset)), num_samples)
        _subset_samplers[num_samples] = SubsetRandomSampler(indices=indices)
    return DataLoader(dataset,
                      sampler=_subset_samplers[num_samples],
                      batch_size=BATCH_SIZE)

# Step 2. Prepare model and dataset

import os
import sys

fp32_model = LightningModel.load_from_checkpoint("models/mobilenet_v3-epoch=192-train_loss=0.01.ckpt").model

fp32_model.eval()

input_shape = (1, 3, 224, 224)
dummy_input = torch.randn(input_shape)

transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

from dataset import DatasetReader
from dataset import utils as dsutils

datasetCOCO = DatasetReader.COCODataset(
    annotation_file = r"/home/centar15-desktop1/LPCV_2025_T1/datasets/coco/annotations/instances_val2017.json", 
    image_dir= r'/home/centar15-desktop1/LPCV_2025_T1/datasets/coco/val2017',
    target_classes=[s.lower() for s in dsutils.GLOBAL_CLASSES],
    transform = transform,
    min_size=60)

eval_dataset = datasetCOCO

print(len(eval_dataset))


/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Total annotations before filtering: 36781
Total annotations after filtering: 8862
Total COCO annotations: 36781
Filtered annotations: 8862
8862


In [4]:

# Step 3. Prepare unlabeled dataset
# NOTE: In the actual use cases, the users should implement this part to serve
#       their own goals if necessary.
class UnlabeledDatasetWrapper(Dataset):
    def __init__(self, dataset):
        self._dataset = dataset

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, index):
        images, _, _ = self._dataset[index]
        return images

unlabeled_dataset = UnlabeledDatasetWrapper(eval_dataset)
unlabeled_data_loader = _create_sampled_data_loader(unlabeled_dataset, CALIBRATION_DATASET_SIZE)

# Step 4. Prepare eval callback
# NOTE: In the actual use cases, the users should implement this part to serve
#       their own goals if necessary.
def eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:
    if num_samples is None:
        num_samples = len(eval_dataset)

    eval_data_loader = _create_sampled_data_loader(eval_dataset, num_samples)

    num_correct_predictions = 0
    for images, labels, _ in eval_data_loader:
        predictions = torch.argmax(model(images.cuda()), dim=1)
        num_correct_predictions += torch.sum(predictions.cpu() == labels)

    return int(num_correct_predictions) / num_samples



In [5]:


# Step 5. Create AutoQuant object
auto_quant = AutoQuant(fp32_model.cuda(),
                       dummy_input.cuda(),
                       unlabeled_data_loader,
                       eval_callback)


In [6]:

# Step 6. (Optional) Set adaround params
ADAROUND_DATASET_SIZE = 2000
adaround_data_loader = _create_sampled_data_loader(unlabeled_dataset, ADAROUND_DATASET_SIZE)
adaround_params = AdaroundParameters(adaround_data_loader, num_batches=len(adaround_data_loader))
# auto_quant.set_adaround_params(adaround_params)


In [ ]:
# print(eval_callback(fp32_model))

0.7932746558338976


In [ ]:
# fp32_model(eval_dataset[0][0])

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [7]:
traced = torch.jit.trace(fp32_model, dummy_input.cuda())

In [8]:

# Step 7. Run AutoQuant
sim, initial_accuracy = auto_quant.run_inference()
print(f"- Quantized Accuracy (before optimization): {initial_accuracy:.4f}")



2025-02-26 17:45:54,288 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.1.block.1.module_mul} 


2025-02-26 17:45:54,289 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.3.module_add} 


2025-02-26 17:45:54,289 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.4.block.2.module_mul_1} 


2025-02-26 17:45:54,290 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.5.block.2.module_mul_2} 


2025-02-26 17:45:54,291 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.5.module_add_1} 


...

2025-02-26 17:47:19,489 - AutoQuant - INFO - Session finished: Batchnorm Folding. (eval score: 0.024035)


- Batchnorm Folding


2025-02-26 17:47:20,521 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-02-26 17:47:20,549 - Quant - INFO - No config file provided, defaulting to config file at /home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json
2025-02-26 17:47:20,561 - Quant - INFO - Unsupported op type Squeeze
2025-02-26 17:47:20,561 - Quant - INFO - Unsupported op type Mean
2025-02-26 17:47:20,569 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


100%|██████████| 63/63 [00:51<00:00,  1.22it/s]

- Quantized Accuracy (before optimization): 0.0240


In [9]:

model, optimized_accuracy, encoding_path = auto_quant.optimize()
print(f"- Quantized Accuracy (after optimization):  {optimized_accuracy:.4f}")

2025-02-26 17:49:29,774 - AutoQuant - INFO - Starting AutoQuant
2025-02-26 17:49:52,134 - AutoQuant - INFO - Target eval score: 0.795193
2025-02-26 17:49:52,135 - AutoQuant - INFO - FP32 eval score (W32A32): 0.795193


- Prepare Model


2025-02-26 17:49:53,063 - Quant - INFO - No config file provided, defaulting to config file at /home/centar15-desktop1/LPCV_2025_T1/.venv310/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json


2025-02-26 17:49:53,074 - Quant - INFO - Unsupported op type Squeeze


2025-02-26 17:49:53,075 - Quant - INFO - Unsupported op type Mean


...

100%|██████████| 63/63 [00:50<00:00,  1.24it/s]


2025-02-26 17:56:31,138 - AutoQuant - INFO - Evaluation finished: W32A8 (eval score: 0.383322)


2025-02-26 17:56:31,139 - AutoQuant - INFO - W32A8 eval score (0.383322) is lower than the target eval score (0.795193). This means it is unlikely that the target eval score can be met using PTQ techniques. Please consider finetuning the model using range learning.


- W32 Evaluation


TypeError: unsupported format string passed to NoneType.__format__

In [19]:
encoding_path